In [1]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.shared import Inches
from docx.oxml.ns import qn
from docx.shared import RGBColor
from docx.shared import Cm
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.table import WD_TABLE_ALIGNMENT,WD_ALIGN_VERTICAL
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

In [2]:
dishi_name = '武汉'

In [3]:
#评估标准插入
biaozhun = pd.read_csv(open('G:/1-规划/评估/软采评估/2019第三季度/使用数据/评估标准_插入.csv',encoding = 'gbk'))

In [4]:
#软采小区替换竞对后数据
ruancai = pd.read_csv(open('G:/1-规划/评估/软采评估/2019第三季度/使用数据/汇总-添评估使用-备份-华为使用邮件数据-tdd-不要大唐.csv',encoding = 'gbk'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#软采栅格
data_merge_quchong = pd.read_csv(open('G:/1-规划/评估/软采评估/2019第三季度/使用数据/第三季度_栅格_添加场景_添加区县_添加弱覆盖1.csv',encoding = 'gbk'))

In [6]:
data_merge_quchong.shape

(3695763, 37)

In [7]:
dishi =['十堰','荆门','武汉','襄阳','黄石','荆州','宜昌','鄂州','孝感','潜江','江汉','黄冈','天门','随州','恩施','咸宁']

In [8]:
document = Document()

In [9]:
# d=Document()
# styles=d.styles
# for s in styles:
#     if s.type==WD_STYLE_TYPE.TABLE:
# #         print(s.name)
# d.save('style.docx')           


In [10]:
def style_heading(wenzi = 'MR竞对覆盖评估体系',jibie = 1,size = 16,ziti = u'微软雅黑',yanse = RGBColor(0,0,0),duiqi = 'CENTER'):
    biaoti = document.add_heading('',jibie)
    if duiqi == 'CENTER':
        biaoti.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif duiqi == 'LEFT':
        biaoti.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT
    else:
        biaoti.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    tt=biaoti.add_run(wenzi)
    tt.font.size = Pt(size)
    tt.bold
    tt.font.name=ziti
    tt._element.rPr.rFonts.set(qn('w:eastAsia'), ziti) 
    tt.font.color.rgb = RGBColor(0,0,0)
    paragraph_format = biaoti.paragraph_format
    paragraph_format.line_spacing = 1.5 # 1.5倍行间距

In [11]:
def style_paragraph(wenzi = '文字描述',size = 10.5,bold = False ,ziti =u'宋体' ):
    p = document.add_paragraph('')
    run1 = p.add_run(wenzi)  # 使用add_run添加文字
    run1.font.size = Pt(size) #字体大小设置，和word里面的字号相对应，小一
    if bold:
        run1.font.bold = True
    run1.font.name=ziti
    run1._element.rPr.rFonts.set(qn('w:eastAsia'), ziti) 
    paragraph_format = p.paragraph_format
    paragraph_format.first_line_indent = Inches(0.25)
    paragraph_format.line_spacing = 1.5 # 1.5倍行间距
def style_paragraph_add(wenzi = '文字描述',size = 10.5,bold = False ,ziti =u'宋体' ):
    run1 = p.add_run(wenzi)  # 使用add_run添加文字
    run1.font.size = Pt(size) #字体大小设置，和word里面的字号相对应，小一
    if bold:
        run1.font.bold = True
    run1.font.name=ziti
    run1._element.rPr.rFonts.set(qn('w:eastAsia'), ziti) 
    paragraph_format = p.paragraph_format
    paragraph_format.first_line_indent = Inches(0.25)
    paragraph_format.line_spacing = 1.5 # 1.5倍行间距

def style_paragraph_hongse_add(wenzi = '文字描述',size = 10.5,bold = False ,ziti =u'宋体' ):
    run1 = p.add_run(wenzi)  # 使用add_run添加文字
    run1.font.size = Pt(size) #字体大小设置，和word里面的字号相对应，小一
    font_color = run1.font
    font_color.color.rgb = RGBColor(255, 0 , 0)
    if bold:
        run1.font.bold = True
    run1.font.name=ziti
    run1._element.rPr.rFonts.set(qn('w:eastAsia'), ziti) 
    paragraph_format = p.paragraph_format
    paragraph_format.first_line_indent = Inches(0.25)
    paragraph_format.line_spacing = 1.5 # 1.5倍行间距
    

## 软采小区-分地市和分区县

In [12]:
ruancai['全网']='全网'

In [13]:
ruancai.shape

(220951, 25)

In [14]:
ruancai.columns

Index(['eutrancell_cgi', '移动覆盖率', '联通覆盖率', '电信覆盖率', 'rsrp_sample_count',
       'rsrp_weak', 'un_maxrsrp_count', 'un_maxrsrp_weak113',
       'te_maxrsrp_count', 'te_maxrsrp_weak113', 'eutrancell_name',
       'region_name', '区县', 'lon', 'lat', '弱覆盖', '有效小区', 'hd', '总问题小区', 'atu',
       '厂家名称', '全网', '制式', '对应栅格区县', '对应栅格地'],
      dtype='object')

In [15]:
def aggfunc(df):
    sum1 = df['rsrp_sample_count'].sum()
    sum2 = df['rsrp_weak'].sum()
    ser = pd.Series([sum1, sum2, 1-sum2/sum1])
    return ser

In [16]:
#各地市指标

In [17]:
data_fgl = ruancai.groupby(by='region_name').apply(aggfunc)
data_fgl.reset_index(inplace=True)
data_fgl.columns = ['全网-指标','移动总点','移动弱点','全网移动覆盖率']
data_fgl.sort_values(by = '全网移动覆盖率',ascending = False,inplace = True)

In [18]:
#添加武汉中兴华为

In [19]:
data_fg_changjia = ruancai.loc[ruancai['region_name']=='武汉'].groupby(by='厂家名称').apply(aggfunc)
data_fg_changjia.reset_index(inplace=True)
data_fg_changjia.columns = ['全网-指标','移动总点','移动弱点','全网移动覆盖率']
data_fg_changjia2 = data_fg_changjia.loc[(data_fg_changjia['全网-指标']=='中兴') | (data_fg_changjia['全网-指标']=='华为')]
data_fg_changjia2.loc[data_fg_changjia2['全网-指标']=='中兴','全网-指标']='武汉中兴'
data_fg_changjia2.loc[data_fg_changjia2['全网-指标']=='华为','全网-指标']='武汉华为'
data_fgl = pd.concat([data_fgl,data_fg_changjia2])

In [20]:
#添加全网指标

In [21]:
data_fg_quanwang = ruancai.groupby(by='全网').apply(aggfunc)
data_fg_quanwang.reset_index(inplace=True)
data_fg_quanwang.columns = ['全网-指标','移动总点','移动弱点','全网移动覆盖率']
data_fgl = pd.concat([data_fgl,data_fg_quanwang])
data_fgl['全网移动覆盖率'] = data_fgl['全网移动覆盖率'].apply(lambda x: format(x, '.2%'))    #Series.apply()让序列的值依次在lambda函数中执行； data['线损率']由小数转化为百分数
data_fgl

,全网-指标,移动总点,移动弱点,全网移动覆盖率
15,黄石,1.084891e+09,26546571.0,97.55%
12,鄂州,8.442823e+08,23283749.0,97.24%
6,武汉,7.735972e+09,244654449.0,96.84%
9,荆州,6.383674e+08,22917236.0,96.41%
8,潜江,2.407243e+08,8918516.0,96.30%
14,黄冈,2.188071e+09,81594967.0,96.27%
4,宜昌,6.104172e+08,23092255.0,96.22%
13,随州,2.506762e+08,9521452.0,96.20%
5,恩施,4.199005e+08,15986054.0,96.19%
10,荆门,2.336347e+08,9395143.0,95.98%


In [22]:
dishi_fgl_qian3=list(data_fgl.iloc[0:3,0])

In [23]:
dishi_fgl_hou3=list(data_fgl.iloc[-6:-3,0])

In [24]:
dishi_fgl_hou3.reverse()

In [25]:
#各区县指标

In [26]:
data_fgl_qx = ruancai.loc[ruancai['region_name']=='武汉'].groupby(by='区县').apply(aggfunc)
data_fgl_qx.reset_index(inplace=True)
data_fgl_qx.columns = ['区域','移动总点','移动弱点','全网移动覆盖率']
data_fgl_qx.sort_values(by = '全网移动覆盖率',ascending = False,inplace = True)
data_fgl_qx['全网移动覆盖率'] = data_fgl_qx['全网移动覆盖率'].apply(lambda x: format(x, '.2%'))   

In [27]:
# data_fgl_qx_ok = data_fgl_qx.loc[data_fgl_qx['武汉-指标'].isin(['武昌','洪山','青山','硚口','东新','江汉','江夏','东西湖','江岸','经开','蔡甸','汉阳','黄陂','新洲'])]

In [28]:
data_fgl_qx['区域'] = data_fgl_qx['区域'].str.replace('区','').str.replace('县','')
data_fgl_qx.loc[data_fgl_qx['区域']=='东湖高新开发','区域']='东新'
data_fgl_qx = data_fgl_qx.drop(data_fgl_qx.loc[data_fgl_qx['区域']=='武汉市','区域'].index)

In [29]:
#武汉指标

In [30]:
data_fgl_wuhan = data_fgl.loc[data_fgl['全网-指标']=='武汉']
data_fgl_wuhan.columns = ['区域','移动总点','移动弱点','全网移动覆盖率']

In [31]:
#区县总计

In [32]:
data_fgl_qx_ok = pd.concat([data_fgl_qx,data_fgl_wuhan])

In [33]:
data_fgl_qx_ok

,区域,移动总点,移动弱点,全网移动覆盖率
9,洪山,1.326621e+09,25264620.0,98.10%
3,武昌,1.218870e+09,28610078.0,97.65%
13,青山,9.367366e+08,25785419.0,97.25%
0,东新,2.312647e+09,73586067.0,96.82%
6,江夏,6.648905e+08,24060144.0,96.38%
7,江岸,1.939194e+08,7612365.0,96.07%
10,硚口,1.357182e+08,5765954.0,95.75%
8,江汉,1.777435e+08,7799626.0,95.61%
5,汉阳,1.473885e+08,7513238.0,94.90%
11,经开,1.309578e+08,6754728.0,94.84%


In [34]:
quxian_fgl_qian3=list(data_fgl_qx_ok.iloc[0:3,0])

In [35]:
quxian_fgl_hou3=list(data_fgl_qx_ok.iloc[-4:-1,0])

In [36]:
quxian_fgl_hou3.reverse()

In [37]:
#添加word内容

In [38]:
style_heading()

In [39]:
style_heading(wenzi ='一、	网络质量覆盖评估',jibie = 2,size = 14 , ziti=u'宋体',duiqi = 'LEFT')

In [40]:
style_paragraph(wenzi = '此用于评估全网网络质量情况，并通过整网、小区、大数据栅格三个维度进行评估分析。')

In [41]:
style_heading(wenzi ='1.整网粒度',jibie = 2,size = 12 , ziti=u'宋体',duiqi = 'LEFT')

In [42]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='全网MR覆盖率：',bold = True)
style_paragraph_add(wenzi='全省分地市，武汉分区县整体MR覆盖率情况。')

In [43]:
':'.join(dishi_fgl_qian3)

'黄石:鄂州:武汉'

In [44]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='指标定义：',bold = True)
style_paragraph_add(wenzi='全网MR覆盖率=所有小区MR RSRP≥-110dbm的采样点总数/采样点总数')
p = document.add_paragraph('')
style_paragraph_add(wenzi='数据来源：',bold = True)
style_paragraph_add(wenzi='本次小区级数据来源于双套系统摸底数据，十堰由于摸底存在问题，采用软采数据。')
p = document.add_paragraph('')
style_paragraph_add(wenzi='2019年第三季度软采数据评估情况：')
p = document.add_paragraph('')
style_paragraph_add(wenzi='全省分地市情况：')
style_paragraph_add(wenzi='、'.join(dishi_fgl_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，')
style_paragraph_add(wenzi='、'.join(dishi_fgl_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差。')

In [45]:
#插入表格并赋值
data = data_fgl.copy()
document.styles['Table Grid'].font.name = u'微软雅黑'
document.styles['Table Grid']._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')

table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [46]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='\n武汉各区县中，')
style_paragraph_add(wenzi='、'.join(quxian_fgl_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，')
style_paragraph_add(wenzi='、'.join(quxian_fgl_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差。')

In [47]:
#插入表格并赋值
data = data_fgl_qx_ok.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [48]:
#问题点统计---2.小区粒度

In [49]:
def aggfunc_wenti(df):
    ruancai_count = df['eutrancell_cgi'].count()
    heidian_count = df['hd'].sum()
    ruofugai_count = df['弱覆盖'].sum()
    wenti_count = df['总问题小区'].sum()
    ser = pd.Series([ruancai_count, heidian_count,ruofugai_count, wenti_count,wenti_count/ruancai_count])
    return ser

In [50]:
#地市问题点
data_wenti = ruancai.groupby(by='region_name').apply(aggfunc_wenti)
data_wenti.reset_index(inplace=True)
data_wenti.columns = ['地市','总小区数','黑点小区数','弱覆盖小区数','问题小区数','问题小区占比']
data_wenti.sort_values(by = '问题小区占比',ascending = True,inplace = True)

In [51]:
#添加武汉中兴华为

In [52]:
data_wenti_changjia = ruancai.loc[ruancai['region_name']=='武汉'].groupby(by='厂家名称').apply(aggfunc_wenti)
data_wenti_changjia.reset_index(inplace=True)
data_wenti_changjia.columns = ['地市','总小区数','黑点小区数','弱覆盖小区数','问题小区数','问题小区占比']
data_wenti_changjia2 = data_wenti_changjia.loc[(data_wenti_changjia['地市']=='中兴') | (data_wenti_changjia['地市']=='华为')]
data_wenti_changjia2.loc[data_wenti_changjia2['地市']=='中兴','地市']='武汉中兴'
data_wenti_changjia2.loc[data_wenti_changjia2['地市']=='华为','地市']='武汉华为'
data_wenti = pd.concat([data_wenti,data_wenti_changjia2])

In [53]:
#添加全网指标

In [54]:
data_wenti_quanwang = ruancai.groupby(by='全网').apply(aggfunc_wenti)
data_wenti_quanwang.reset_index(inplace=True)
data_wenti_quanwang.columns = ['地市','总小区数','黑点小区数','弱覆盖小区数','问题小区数','问题小区占比']
data_wenti = pd.concat([data_wenti,data_wenti_quanwang])
data_wenti['问题小区占比'] = data_wenti['问题小区占比'].apply(lambda x: format(x, '.2%'))    #Series.apply()让序列的值依次在lambda函数中执行； data['线损率']由小数转化为百分数
data_wenti

,地市,总小区数,黑点小区数,弱覆盖小区数,问题小区数,问题小区占比
15,黄石,9829.0,357.0,677.0,810.0,8.24%
12,鄂州,4901.0,204.0,366.0,431.0,8.79%
0,十堰,5719.0,289.0,400.0,523.0,9.14%
9,荆州,20412.0,425.0,2152.0,2272.0,11.13%
8,潜江,3116.0,171.0,322.0,373.0,11.97%
4,宜昌,19589.0,606.0,2334.0,2545.0,12.99%
6,武汉,55873.0,3602.0,6005.0,8108.0,14.51%
7,江汉,3544.0,306.0,480.0,565.0,15.94%
3,孝感,13209.0,998.0,2057.0,2249.0,17.03%
11,襄阳,21299.0,1065.0,3314.0,3686.0,17.31%


In [55]:
dishi_wenti_qian3=list(data_wenti.iloc[0:3,0])

In [56]:
dishi_wenti_hou3 = list(data_wenti.iloc[-6:-3,0])

In [57]:
dishi_wenti_hou3.reverse()

In [58]:
#武汉问题点

In [59]:
data_wenti_qx = ruancai.loc[ruancai['region_name']=='武汉'].groupby(by='区县').apply(aggfunc_wenti)
data_wenti_qx.reset_index(inplace=True)
data_wenti_qx.columns = ['区县','总小区数','黑点小区数','弱覆盖小区数','问题小区数','问题小区占比']
data_wenti_qx.sort_values(by = '问题小区占比',ascending = True,inplace = True)
data_wenti_qx['问题小区占比'] = data_wenti_qx['问题小区占比'].apply(lambda x: format(x, '.2%'))   

In [60]:
#data_wenti_qx_ok = data_wenti_qx.loc[data_wenti_qx['区县'].isin(['武昌','洪山','青山','硚口','东新','江汉','江夏','东西湖','江岸','经开','蔡甸','汉阳','黄陂','新洲'])]

In [61]:
#武汉指标

In [62]:
data_wenti_wuhan = data_wenti.loc[data_wenti['地市']=='武汉']
data_wenti_wuhan.columns = ['区县','总小区数','黑点小区数','弱覆盖小区数','问题小区数','问题小区占比']

In [63]:
data_wenti_qx

,区县,总小区数,黑点小区数,弱覆盖小区数,问题小区数,问题小区占比
9,洪山区,4250.0,78.0,132.0,165.0,3.88%
3,武昌区,4157.0,100.0,150.0,208.0,5.00%
13,青山区,4711.0,189.0,209.0,285.0,6.05%
0,东湖高新开发区,7805.0,446.0,539.0,726.0,9.30%
7,江岸区,4402.0,335.0,353.0,609.0,13.83%
6,江夏区,3497.0,220.0,459.0,513.0,14.67%
10,硚口区,3057.0,243.0,285.0,455.0,14.88%
8,江汉区,4354.0,431.0,412.0,753.0,17.29%
5,汉阳区,3486.0,217.0,467.0,607.0,17.41%
12,蔡甸区,2137.0,95.0,344.0,395.0,18.48%


In [64]:
data_wenti_qx['区县'] = data_wenti_qx['区县'].str.replace('区','').str.replace('县','')
data_wenti_qx.loc[data_wenti_qx['区县']=='东湖高新开发','区县']='东新'
data_wenti_qx = data_wenti_qx.drop(data_wenti_qx.loc[data_wenti_qx['区县']=='武汉市','区县'].index)

In [65]:
#区县总计

In [66]:
data_wenti_qx_ok = pd.concat([data_wenti_qx,data_wenti_wuhan])

In [67]:
data_wenti_qx_ok

,区县,总小区数,黑点小区数,弱覆盖小区数,问题小区数,问题小区占比
9,洪山,4250.0,78.0,132.0,165.0,3.88%
3,武昌,4157.0,100.0,150.0,208.0,5.00%
13,青山,4711.0,189.0,209.0,285.0,6.05%
0,东新,7805.0,446.0,539.0,726.0,9.30%
7,江岸,4402.0,335.0,353.0,609.0,13.83%
6,江夏,3497.0,220.0,459.0,513.0,14.67%
10,硚口,3057.0,243.0,285.0,455.0,14.88%
8,江汉,4354.0,431.0,412.0,753.0,17.29%
5,汉阳,3486.0,217.0,467.0,607.0,17.41%
12,蔡甸,2137.0,95.0,344.0,395.0,18.48%


In [68]:
quxian_wenti_qian3 = list(data_wenti_qx_ok.iloc[0:3,0])

In [69]:
quxian_wenti_hou3 = list(data_wenti_qx_ok.iloc[-4:-1,0])

In [70]:
quxian_wenti_hou3.reverse()

In [71]:
style_heading(wenzi ='2.小区粒度',jibie = 2,size = 12 , ziti=u'宋体',duiqi = 'LEFT')

In [72]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='MR覆盖问题小区占比：',bold = True)
style_paragraph_add(wenzi='全省分地市，武汉分区县MR覆盖问题小区占比情况')

In [73]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='指标定义：',bold = True)
style_paragraph_add(wenzi='MR覆盖问题小区占比=（小区级MR覆盖率<90%或MR竞对黑点有效小区）/总小区数')
p = document.add_paragraph('')
style_paragraph_add(wenzi='数据来源：',bold = True)
style_paragraph_add(wenzi='本次小区级数据来源于双套系统摸底数据，十堰由于摸底存在问题，采用软采数据。')
p = document.add_paragraph('')
style_paragraph_add(wenzi='全省分地市情况：')
style_paragraph_add(wenzi='、'.join(dishi_wenti_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR问题小区较少。')
style_paragraph_add(wenzi='、'.join(dishi_wenti_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR问题小区较多。')

In [74]:
#插入表格并赋值
data = data_wenti.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [75]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='\n武汉各区县中，')
style_paragraph_add(wenzi='、'.join(quxian_wenti_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR问题小区较少。')
style_paragraph_add(wenzi='、'.join(quxian_wenti_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR问题小区较多。')

In [76]:
#插入表格并赋值
data = data_wenti_qx_ok.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

## 软采栅格-武汉区域分区县

In [77]:
#3.大数据栅格粒度

In [78]:
# data_merge_quchong = pd.read_csv(open('G:\\1-规划\\评估\\软采评估\\2019第三季度\\软采数据\\软采栅格\\第三季度_栅格_添加场景_添加区县_添加弱覆盖.csv',encoding = 'gbk'))

In [79]:
# data_merge_quchong['全网'] ="全网"
# data_merge_quchong['fgl'] = 1-data_merge_quchong['a.rsrp_weak']/data_merge_quchong['a.rsrp_sample_count']
# data_merge_quchong['大于100采样点'] = 0
# data_merge_quchong.loc[data_merge_quchong['a.rsrp_sample_count']>100,'大于100采样点']=1
# data_merge_quchong['弱覆盖栅格'] = 0
# data_merge_quchong.loc[data_merge_quchong['fgl']<0.936,'弱覆盖栅格']=1
# data_merge_quchong['严重弱覆盖栅格'] = 0
# data_merge_quchong.loc[data_merge_quchong['fgl']<0.7,'严重弱覆盖栅格']=1

In [80]:
def aggfunc_grid(df):
    grid_count = df['a.grid_id'].count()
    grid_100_count = df['大于100采样点'].sum()
    df1 = df.loc[df['大于100采样点']==1]
    grid_fgl936_count = df1['弱覆盖栅格'].sum()
    ser = pd.Series([grid_count, grid_100_count,grid_fgl936_count, grid_fgl936_count/grid_100_count])
    return ser

In [81]:
data_merge_quchong.columns

Index(['a.grid_id', 'a.day', 'a.grid_leftbottom_latitude',
       'a.grid_leftbottom_longitude', 'a.grid_lefttop_latitude',
       'a.grid_lefttop_longitude', 'a.grid_rightbottom_latitude',
       'a.grid_rightbottom_longitude', 'a.grid_righttop_latitude',
       'a.grid_righttop_longitude', 'a.inter_maxrsrp_weak', 'a.region_name',
       'a.rsrp_sample_count', 'a.rsrp_sum', 'a.rsrp_weak', 'a.scrsrp_110',
       'a.scrsrp_110_95', 'a.scrsrp_95', 'a.te_maxrsrp_sample_count',
       'a.te_maxrsrp_sum', 'a.te_maxrsrp_weak', 'a.te_maxrsrp_weak_l2',
       'a.un_maxrsrp_sample_count', 'a.un_maxrsrp_sum', 'a.un_maxrsrp_weak',
       'a.un_maxrsrp_weak_l2', 'b.dl_data', 'b.ul_data', 'lon', 'lat', '区域类型',
       '区县', '全网', 'fgl', '大于100采样点', '弱覆盖栅格', '严重弱覆盖栅格'],
      dtype='object')

In [82]:
#地市问题点

In [83]:
data_grid = data_merge_quchong.groupby(by='a.region_name').apply(aggfunc_grid)
data_grid.reset_index(inplace=True)
data_grid.columns = ['19年9月','总栅格','大于100采样点','弱覆盖栅格数量','大数据弱覆盖栅格占比']
data_grid.sort_values(by = '大数据弱覆盖栅格占比',ascending = True,inplace = True)

In [84]:
#添加全网指标

In [85]:
data_grid_quanwang = data_merge_quchong.groupby(by='全网').apply(aggfunc_grid)
data_grid_quanwang.reset_index(inplace=True)
data_grid_quanwang.columns = ['19年9月','总栅格','大于100采样点','弱覆盖栅格数量','大数据弱覆盖栅格占比']
data_grid = pd.concat([data_grid,data_grid_quanwang])
data_grid['大数据弱覆盖栅格占比'] = data_grid['大数据弱覆盖栅格占比'].apply(lambda x: format(x, '.2%'))    #Series.apply()让序列的值依次在lambda函数中执行； data['线损率']由小数转化为百分数
data_grid

,19年9月,总栅格,大于100采样点,弱覆盖栅格数量,大数据弱覆盖栅格占比
9,荆州,17987.0,479.0,69.0,14.41%
10,荆门,212900.0,38159.0,6621.0,17.35%
4,宜昌,329050.0,73981.0,19954.0,26.97%
13,随州,189961.0,44059.0,11968.0,27.16%
11,襄阳,709892.0,202800.0,70029.0,34.53%
5,恩施,258106.0,41465.0,15495.0,37.37%
1,咸宁,187005.0,34156.0,18037.0,52.81%
15,黄石,154987.0,99528.0,56981.0,57.25%
6,武汉,488445.0,277306.0,167319.0,60.34%
8,潜江,83759.0,45252.0,28840.0,63.73%


In [86]:
grid_zhanbi_qian3 = list(data_grid.iloc[0:3,0])

In [87]:
grid_zhanbi_hou3 = list(data_grid.iloc[-4:-1,0])

In [88]:
grid_zhanbi_hou3.reverse()

In [89]:
#区县问题点

In [90]:
data_grid_qx = data_merge_quchong.loc[data_merge_quchong['a.region_name']=='武汉'].groupby(by='区县').apply(aggfunc_grid)
data_grid_qx.reset_index(inplace=True)
data_grid_qx.columns = ['19年9月','总栅格','大于100采样点','弱覆盖栅格数量','大数据弱覆盖栅格占比']
data_grid_qx.sort_values(by = '大数据弱覆盖栅格占比',ascending = True,inplace = True)
data_grid_qx['大数据弱覆盖栅格占比'] = data_grid_qx['大数据弱覆盖栅格占比'].apply(lambda x: format(x, '.2%'))   
data_grid_qx_ok = data_grid_qx.loc[data_grid_qx['19年9月'].isin(['武昌','洪山','青山','硚口','东新','江汉','江夏','东西湖','江岸','经开','蔡甸','汉阳','黄陂','新洲'])]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  


In [91]:
#武汉指标

In [92]:
data_grid_wuhan = data_grid.loc[data_grid['19年9月']=='武汉']
data_grid_wuhan.columns = ['19年9月','总栅格','大于100采样点','弱覆盖栅格数量','大数据弱覆盖栅格占比']
data_grid_qx_ok = pd.concat([data_grid_qx_ok,data_grid_wuhan])
data_grid_qx_ok

,19年9月,总栅格,大于100采样点,弱覆盖栅格数量,大数据弱覆盖栅格占比
26,青山,23041.0,17041.0,8212.0,48.19%
25,蔡甸,53949.0,28792.0,14684.0,51.00%
15,武昌,4939.0,4922.0,2563.0,52.07%
22,硚口,3879.0,3742.0,1991.0,53.21%
2,东西湖,32227.0,20522.0,11021.0,53.70%
24,经开,29051.0,16568.0,8967.0,54.12%
19,江岸,6765.0,6369.0,3539.0,55.57%
21,洪山,10068.0,9379.0,5345.0,56.99%
1,东新,30269.0,24797.0,14291.0,57.63%
17,汉阳,7348.0,6805.0,4030.0,59.22%


In [93]:
grid_quxian_zhanbi_qian3 = list(data_grid_qx_ok.iloc[0:3,0])

In [94]:
grid_quxian_zhanbi_hou3 = list(data_grid_qx_ok.iloc[-4:-1,0])

In [95]:
grid_quxian_zhanbi_hou3.reverse()

In [96]:
style_heading(wenzi ='3.大数据栅格粒度',jibie = 2,size = 12 , ziti=u'宋体',duiqi = 'LEFT')

In [97]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='1)大数据MR问题栅格占比:',bold = True)
style_paragraph_add(wenzi='全省分地市，武汉分区县大数据问题栅格占比情况')
p = document.add_paragraph('')
style_paragraph_add(wenzi='指标定义：',bold = True)
style_paragraph_add(wenzi='大数据MR问题栅格占比=大数据问题栅格数/总有效栅格数')
p = document.add_paragraph('')
style_paragraph_add(wenzi='大数据问题栅格：有效栅格MR覆盖率小于93.6%(此处门限定义为目标网设定全网达到96%的换算值)')
p = document.add_paragraph('')
style_paragraph_add(wenzi='总有效栅格数：栅格采样点数大于100')
p = document.add_paragraph('')
style_paragraph_add(wenzi='全省分地市情况：')
style_paragraph_add(wenzi='、'.join(grid_zhanbi_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR弱覆盖栅格较少。')
style_paragraph_add(wenzi='、'.join(grid_zhanbi_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR弱覆盖栅格较多。')
p = document.add_paragraph('')
style_paragraph_hongse_add(wenzi='注：大数据问题栅格标准是栅格覆盖率小于93.6%，故此处指标较高。')


In [98]:
#插入表格并赋值
data = data_grid.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [99]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='\n武汉各区县中，')
style_paragraph_add(wenzi='、'.join(grid_quxian_zhanbi_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR弱覆盖栅格较少。')
style_paragraph_add(wenzi='、'.join(grid_quxian_zhanbi_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR弱覆盖栅格较多。')

In [100]:
#插入表格并赋值
data = data_grid_qx_ok.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

### 严重弱覆盖栅格占比 

In [101]:
def aggfunc_grid_yanzhong(df):
    grid_count = df['a.grid_id'].count()
    grid_100_count = df['大于100采样点'].sum()
    df1 = df.loc[df['大于100采样点']==1]
    grid_fgl936_count = df1['严重弱覆盖栅格'].sum()
    ser = pd.Series([grid_count, grid_100_count,grid_fgl936_count, grid_fgl936_count/grid_100_count])
    return ser

In [102]:
#地市问题点

In [103]:
data_grid_yz = data_merge_quchong.groupby(by='a.region_name').apply(aggfunc_grid_yanzhong)
data_grid_yz.reset_index(inplace=True)
data_grid_yz.columns = ['19年9月','总栅格','大于100采样点','严重弱覆盖栅格数量','严重弱覆盖栅格占比']
data_grid_yz.sort_values(by = '严重弱覆盖栅格占比',ascending = True,inplace = True)

In [104]:
#添加全网指标

In [105]:
data_grid_quanwang_yz = data_merge_quchong.groupby(by='全网').apply(aggfunc_grid_yanzhong)
data_grid_quanwang_yz.reset_index(inplace=True)
data_grid_quanwang_yz.columns = ['19年9月','总栅格','大于100采样点','严重弱覆盖栅格数量','严重弱覆盖栅格占比']
data_grid_yz = pd.concat([data_grid_yz,data_grid_quanwang_yz])
data_grid_yz['严重弱覆盖栅格占比'] = data_grid_yz['严重弱覆盖栅格占比'].apply(lambda x: format(x, '.2%'))    #Series.apply()让序列的值依次在lambda函数中执行； data['线损率']由小数转化为百分数
data_grid_yz

,19年9月,总栅格,大于100采样点,严重弱覆盖栅格数量,严重弱覆盖栅格占比
9,荆州,17987.0,479.0,2.0,0.42%
10,荆门,212900.0,38159.0,1874.0,4.91%
4,宜昌,329050.0,73981.0,5786.0,7.82%
13,随州,189961.0,44059.0,4399.0,9.98%
11,襄阳,709892.0,202800.0,30651.0,15.11%
8,潜江,83759.0,45252.0,7060.0,15.60%
1,咸宁,187005.0,34156.0,5747.0,16.83%
5,恩施,258106.0,41465.0,7629.0,18.40%
12,鄂州,77605.0,47373.0,9790.0,20.67%
15,黄石,154987.0,99528.0,20969.0,21.07%


In [106]:
grid_yanzhong_qian3 = list(data_grid_yz.iloc[0:3,0])

In [107]:
grid_yanzhong_hou3 = list(data_grid_yz.iloc[-4:-1,0])

In [108]:
grid_yanzhong_hou3.reverse()

In [109]:
#区县问题点

In [110]:
data_grid_qx_yz = data_merge_quchong.loc[data_merge_quchong['a.region_name']=='武汉'].groupby(by='区县').apply(aggfunc_grid_yanzhong)
data_grid_qx_yz.reset_index(inplace=True)
data_grid_qx_yz.columns = ['19年9月','总栅格','大于100采样点','严重弱覆盖栅格数量','严重弱覆盖栅格占比']
data_grid_qx_yz.sort_values(by = '严重弱覆盖栅格占比',ascending = True,inplace = True)
data_grid_qx_yz['严重弱覆盖栅格占比'] = data_grid_qx_yz['严重弱覆盖栅格占比'].apply(lambda x: format(x, '.2%'))   
data_grid_qx_yz_ok = data_grid_qx_yz.loc[data_grid_qx_yz['19年9月'].isin(['武昌','洪山','青山','硚口','东新','江汉','江夏','东西湖','江岸','经开','蔡甸','汉阳','黄陂','新洲'])]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  


In [111]:
#武汉指标

In [112]:
data_grid_wuhan_yz = data_grid_yz.loc[data_grid_yz['19年9月']=='武汉']
data_grid_wuhan_yz.columns = ['19年9月','总栅格','大于100采样点','严重弱覆盖栅格数量','严重弱覆盖栅格占比']
data_grid_qx_yz_ok = pd.concat([data_grid_qx_yz_ok,data_grid_wuhan_yz])
data_grid_qx_yz_ok

,19年9月,总栅格,大于100采样点,严重弱覆盖栅格数量,严重弱覆盖栅格占比
15,武昌,4939.0,4922.0,215.0,4.37%
21,洪山,10068.0,9379.0,1169.0,12.46%
22,硚口,3879.0,3742.0,474.0,12.67%
26,青山,23041.0,17041.0,2410.0,14.14%
1,东新,30269.0,24797.0,3538.0,14.27%
20,江汉,4013.0,3799.0,563.0,14.82%
19,江岸,6765.0,6369.0,970.0,15.23%
17,汉阳,7348.0,6805.0,1304.0,19.16%
25,蔡甸,53949.0,28792.0,7406.0,25.72%
24,经开,29051.0,16568.0,4264.0,25.74%


In [113]:
grid_yanzhong_quxian_qian3 = list(data_grid_qx_yz_ok.iloc[0:3,0])

In [114]:
grid_yanzhong_quxian_hou3 = list(data_grid_qx_yz_ok.iloc[-4:-1,0])

In [115]:
grid_yanzhong_quxian_hou3.reverse()

In [116]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='\n2)严重弱覆盖栅格占比：',bold = True)
style_paragraph_add(wenzi='全省分地市，武汉分区县大数据问题栅格占比情况')
p = document.add_paragraph('')
style_paragraph_add(wenzi='指标定义：',bold = True)
style_paragraph_add(wenzi='大数据严重弱覆盖栅格占比=大数据严重问题栅格数/总有效栅格数大数据严重问题栅格：有效栅格MR覆盖率小于70%.')

p = document.add_paragraph('')
style_paragraph_add(wenzi='全省分地市情况：')
style_paragraph_add(wenzi='、'.join(grid_yanzhong_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR弱覆盖栅格较少。')
style_paragraph_add(wenzi='、'.join(grid_yanzhong_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR弱覆盖栅格较多。')

In [117]:
#插入表格并赋值
data = data_grid_yz.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [118]:
p = document.add_paragraph('')
style_paragraph_add(wenzi='\n武汉各区县中，')
style_paragraph_add(wenzi='、'.join(grid_yanzhong_quxian_qian3),bold = True)
style_paragraph_add(wenzi='覆盖较好，MR严重弱覆盖栅格较少。')
style_paragraph_add(wenzi='、'.join(grid_yanzhong_quxian_hou3),bold = True)
style_paragraph_add(wenzi='覆盖相对较差，MR严重弱覆盖栅格较多。')

In [119]:
#插入表格并赋值
data = data_grid_qx_yz_ok.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [120]:
#添加word内容=======================
style_heading(wenzi ='二、	整体评估',jibie = 2,size = 14 , ziti=u'宋体',duiqi = 'LEFT')
style_paragraph(wenzi = '各指标项分值见分值列：地市指标按指标情况线性取值。')

#插入表格并赋值
data = biaozhun.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
#得分计算——地市
list_dishi2 = ['孝感','襄阳','咸宁','随州','十堰','荆州','荆门','黄石','鄂州','黄冈','恩施','宜昌','潜江','天门','江汉','武汉','全网']
pingfen_dishi = pd.DataFrame({'地市': list_dishi2})
pingfen_dishi['全网MR覆盖率']=0
pingfen_dishi['得分1']=0
pingfen_dishi['问题小区占比']=0
pingfen_dishi['得分2']=0
pingfen_dishi['大数据MR问题栅格占比']=0
pingfen_dishi['得分3']=0
pingfen_dishi['严重弱覆盖栅格占比']=0
pingfen_dishi['得分4']=0
pingfen_dishi['总得分']=0

pingfen_dishi.set_index('地市',inplace = True)
data_fgl_index = data_fgl.set_index('全网-指标')
data_wenti_index = data_wenti.set_index('地市')
data_grid_index = data_grid.set_index('19年9月')
data_grid_yz_index = data_grid_yz.set_index('19年9月')
pingfen_dishi['全网MR覆盖率'].update(data_fgl_index['全网移动覆盖率'])
pingfen_dishi['问题小区占比'].update(data_wenti_index['问题小区占比'])
pingfen_dishi['大数据MR问题栅格占比'].update(data_grid_index['大数据弱覆盖栅格占比'])
pingfen_dishi['严重弱覆盖栅格占比'].update(data_grid_yz_index['严重弱覆盖栅格占比'])
pingfen_dishi['得分1'] = pd.to_numeric(pingfen_dishi['全网MR覆盖率'].str.replace('%',''))*20/100
pingfen_dishi['得分2'] = 10-pd.to_numeric(pingfen_dishi['问题小区占比'].str.replace('%',''))*10/100
pingfen_dishi['得分3'] = 10-pd.to_numeric(pingfen_dishi['大数据MR问题栅格占比'].str.replace('%',''))*10/100
pingfen_dishi['得分4'] = 60-pd.to_numeric(pingfen_dishi['严重弱覆盖栅格占比'].str.replace('%',''))/100*60
pingfen_dishi['总得分'] = pingfen_dishi['得分1']+pingfen_dishi['得分2']+pingfen_dishi['得分3']+pingfen_dishi['得分4']
pingfen_dishi_1 = pingfen_dishi.round(decimals = 2)
pingfen_dishi_1.sort_values('总得分',ascending=False,inplace = True)
pingfen_dishi_1.reset_index(inplace = True)

pingfen_dishi_res = pd.concat([pingfen_dishi_1.loc[pingfen_dishi_1['地市']!='全网'],pingfen_dishi_1.loc[pingfen_dishi_1['地市']=='全网']])
pingfen_dishi_res.reset_index(drop=True,inplace = True)

pingfen_dishi_res.shape
if data_grid_qx_yz_ok.shape[0]>=7:
    pingfen_dishi_qian3 = list(pingfen_dishi_res.iloc[0:3,0])
    pingfen_dishi_hou3 = list(pingfen_dishi_res.iloc[-4:-1,0])
    pingfen_dishi_hou3.reverse()
elif pingfen_dishi_res.shape[0]>=5:
    pingfen_dishi_qian3 = list(pingfen_dishi_res.iloc[0:2,0])
    pingfen_dishi_hou3 = list(pingfen_dishi_res.iloc[-3:-1,0])
    pingfen_dishi_hou3.reverse()
else:
    pingfen_dishi_qian3 = list(pingfen_dishi_res.iloc[0:1,0])
    pingfen_dishi_hou3 = list(pingfen_dishi_res.iloc[-2:-1,0])
    pingfen_dishi_hou3.reverse()
wuhan_paiming = pingfen_dishi_res[pingfen_dishi_res['地市'].isin(['武汉'])].index[0]+1

p = document.add_paragraph('')
p = document.add_paragraph('')
style_paragraph_add(wenzi='最后依据地市每项得分的总和得到地市总得分，排序情况如下：')
style_paragraph_add(wenzi='、'.join(pingfen_dishi_qian3),bold = True)
style_paragraph_add(wenzi='得分最高。')
style_paragraph_add(wenzi='、'.join(pingfen_dishi_hou3),bold = True)
style_paragraph_add(wenzi='得分最低。')
style_paragraph_hongse_add(wenzi='武分排名'+str(wuhan_paiming)+'。')
#插入表格并赋值
data = pingfen_dishi_res.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [122]:
#得分-区县
pingfen_quxian = data_fgl_qx_ok[['区域','全网移动覆盖率']]
pingfen_quxian = pingfen_quxian.reindex(columns=['区域','全网移动覆盖率','得分1','问题小区占比','得分2','大数据MR问题栅格占比','得分3','严重弱覆盖栅格占比','得分4','总得分'])
pingfen_quxian1 = pingfen_quxian.set_index('区域')
data_wenti_qx_ok_index = data_wenti_qx_ok.set_index('区县')
data_grid_qx_ok_index = data_grid_qx_ok.set_index('19年9月')
data_grid_qx_yz_ok_index = data_grid_qx_yz_ok.set_index('19年9月')
pingfen_quxian1['问题小区占比'].update(data_wenti_qx_ok_index['问题小区占比'])
pingfen_quxian1['大数据MR问题栅格占比'].update(data_grid_qx_ok_index['大数据弱覆盖栅格占比'])
pingfen_quxian1['严重弱覆盖栅格占比'].update(data_grid_qx_yz_ok_index['严重弱覆盖栅格占比'])
pingfen_quxian1['得分1'] = pd.to_numeric(pingfen_quxian1['全网移动覆盖率'].str.replace('%',''))*20/100
pingfen_quxian1['得分2'] = 10-pd.to_numeric(pingfen_quxian1['问题小区占比'].str.replace('%',''))*10/100
pingfen_quxian1['得分3'] = 10-pd.to_numeric(pingfen_quxian1['大数据MR问题栅格占比'].str.replace('%',''))*10/100
pingfen_quxian1['得分4'] = 60-pd.to_numeric(pingfen_quxian1['严重弱覆盖栅格占比'].str.replace('%',''))/100*60
pingfen_quxian1['总得分'] = pingfen_quxian1['得分1']+pingfen_quxian1['得分2']+pingfen_quxian1['得分3']+pingfen_quxian1['得分4']
pingfen_quxian_1 = pingfen_quxian1.round(decimals = 2)
pingfen_quxian_1.sort_values('总得分',ascending=False,inplace = True)
pingfen_quxian_1.reset_index(inplace = True)

pingfen_quxian_res = pd.concat([pingfen_quxian_1.loc[pingfen_quxian_1['区域']!=dishi_name],pingfen_quxian_1.loc[pingfen_quxian_1['区域']==dishi_name]])
pingfen_quxian_res.reset_index(drop=True,inplace = True)

if pingfen_quxian_res.shape[0]>=7:
    pingfen_quxian_qian3 = list(pingfen_quxian_res.iloc[0:3,0])
    pingfen_quxian_hou3 = list(pingfen_quxian_res.iloc[-4:-1,0])
    pingfen_quxian_hou3.reverse()
elif pingfen_quxian_res.shape[0]>=5:
    pingfen_quxian_qian3 = list(pingfen_quxian_res.iloc[0:2,0])
    pingfen_quxian_hou3 = list(pingfen_quxian_res.iloc[-3:-1,0])
    pingfen_quxian_hou3.reverse()
else:
    pingfen_quxian_qian3 = list(pingfen_quxian_res.iloc[0:1,0])
    pingfen_quxian_hou3 = list(pingfen_quxian_res.iloc[-2:-1,0])
    pingfen_quxian_hou3.reverse()

p = document.add_paragraph('')
p = document.add_paragraph('')
style_paragraph_add(wenzi='依据'+dishi_name+'区县的每项得分总和得到各区县的总得分，排序如下：')
if pingfen_quxian_res.shape[0]<3:
    pass
else:
    style_paragraph_add(wenzi='、'.join(pingfen_quxian_qian3),bold = True)
    style_paragraph_add(wenzi='得分最高。')
    style_paragraph_add(wenzi='、'.join(pingfen_quxian_hou3),bold = True)
    style_paragraph_add(wenzi='得分最低。')
#插入表格并赋值
data = pingfen_quxian_res.copy()
table = document.add_table(rows = data.shape[0]+1 , cols = data.shape[1],style = 'Table Grid')
table.autofit=True
table.rows[0].height=Cm(1)
# 列名赋值
hdr_cells = table.rows[0].cells
for row in list(data.columns):
    for num in range(0, data.shape[1]):
        hdr_cells[num].text = u'' + list(data.columns)[num]
        table.cell(0,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(0,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading_elm_2 = parse_xml(r'<w:shd {} w:fill="#BFBFBF"/>'.format(nsdecls('w')))
        table.cell(0,num)._tc.get_or_add_tcPr().append(shading_elm_2)
    break
# 内容赋值
for row in range(1,data.shape[0]+1):
    row_cells = table.rows[row].cells
    for num in range(0, data.shape[1]):
        row_cells[num].text = u'' + str(data.iloc[row-1,num])
        table.cell(row,num).paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,num).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

In [123]:
document.save('ceshi1.docx')  #保存文档